## Lee TH., Seregina E.: "Combining Forecasts under Structural Breaks Using Graphical LASSO"

### This Python notebook can be used to reproduce Monte Carlo results for forecast combination weights and precision matrix in Supplementary Appendix Figures E1, E3, E5, E6, E8

In [ ]:
import sys
print(sys.version)

In [ ]:
%pip install regain==0.3.9

#### Please make sure to place "GL.py" and "TVGL.py" in the same directory as this notebook

In [ ]:
from __future__ import division
from GL import GraphicalLasso
from TVGL2 import TimeGraphicalLasso
from regain.datasets import make_dataset
from regain.utils import error_norm_time
import numpy as np 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy
import scipy.linalg   # SciPy Linear Algebra Library
import pandas as pd
import statistics
import math
from scipy.linalg import cholesky
import sklearn
from sklearn import datasets
from numpy.linalg import inv
from sklearn.covariance import GraphicalLassoCV
import warnings
warnings.filterwarnings("ignore")
from numpy import savetxt

In [ ]:
##############FUNCTIONS#############
def portfolios(X,estsigm):
    # X = X.to_numpy()
    mu = np.mean(X,axis=0).reshape(X.shape[1],1)
    p = len(mu) 
    one = np.ones([p,1])
    phi = one.T@estsigm@one
    #GMV##
    gmv = (estsigm @ one) / phi 
    return [gmv]  
#########################################

def CV_theta(penalty, Y, breaks, betas, covariate, alpha_set, beta_set, isig, denominator): #X is returns, Y is residuals, K=k,
#################SFEs ARE HERE##################
    errTheta=np.zeros([len(alpha_set),len(beta_set)])
    for alpha in range(0,len(alpha_set)):
        for beta in range(0,len(beta_set)):
            if penalty == '':
                tvfgl = TimeGraphicalLasso(max_iter=100, alpha = alpha_set[alpha], beta = beta_set[beta]).fit(Y, breaks)
            else:    
                tvfgl = TimeGraphicalLasso(psi=penalty, max_iter=100, alpha = alpha_set[alpha], beta = beta_set[beta]).fit(Y, breaks)
            if k==1:
                Thetatvfgl=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                Thetatvfgl=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]

            errTheta[alpha,beta] = math.log2(np.linalg.norm(Thetatvfgl-isig, ord=2)/np.sqrt(denominator))
    [alphaopt, betaopt] = [alpha_set[np.where(errTheta == np.min(errTheta))[0]], beta_set[np.where(errTheta == np.min(errTheta))[1]]]      
    return np.array([alphaopt,betaopt])


def CV_gamma(gamma_set,r, r1, r2, k):
    err_gamma=np.zeros([len(gamma_set),1])
    err_loo=np.zeros([r2.shape[0],1])    
    for gamma in range(0,len(gamma_set)):
        for loo in range(0,r2.shape[0]):
            # r = F@Lambda[0:kDGP,] + eps
            r1_cv = r1*gamma_set[gamma]
            r2_cv = np.delete(r2, loo, axis=0)
            ##leaving one time -series out
            r_cv = np.concatenate((r1_cv, r2_cv), axis=0)

            ##estimating factors and loadings with LOO returns           
            L, V = np.linalg.eigh(np.dot(r_cv.T, r_cv))
            idx = L.argsort()[::-1]
            L = L[idx]  # eigenvalues, Nx1
            V = V[:, idx]  # eigenvectors columns, NxN
            lmb = V[:, 0:k]  # kx1
            Fhat = np.dot(r_cv, lmb)  # Txr (r=1 for PC1)
            sum_i = np.zeros([r_cv.shape[1],1])
            for i in range(r.shape[1]):
                sum_j = np.zeros([r2.shape[0],1])
                ##computing sum of squared errors for the post-break period
                for j in range(0,r2.shape[0]):
                    sum_j[j] = (r2[j,i]-Fhat[j,:]@lmb.T[:,i])**2
                sum_i[i] = np.sum(sum_j) 
            ##collecting SSEs for all LOOs    
            err_loo[loo] = np.sum(sum_i)/(r.shape[1]*(r2.shape[0]))   
        ##compute average of all LOOs for one gamma                
        err_gamma[gamma] = np.sum(err_loo)
    return  gamma_set[np.where(err_gamma == np.min(err_gamma))[0]]

In [ ]:
###PARAMETERS
gamma_opt_zero = 0
gamma_set = np.arange(0,1.05,0.05)  
alpha_set = np.array([0, 0.25, 0.5, 1, 10, 30]).astype(float)
beta_set = np.array([0, 0.25, 0.5, 1, 10, 30]).astype(float)
sigmaF = 1
rho=0.9 #was 0.2 before IJF revision
phi = 0.2
ite=500
# sample_size= np.array([int(round(2**(7),0)),int(round(2**(7.5),0))])
###Modifying sample size 
# sample_size= np.array([int(round(2**(9.5),0))])
#initial one (full)
sample_size= np.array([int(round(2**(7),0)),int(round(2**(7.5),0)),int(round(2**(8),0)),int(round(2**(8.5),0)),int(round(2**(9),0)), int(round(2**(9.5),0))])

In [ ]:
cum_errWeight_GL = np.zeros((len(sample_size),1))
cum_errWeight_FGL = np.zeros((len(sample_size),1))
cum_errWeight_EW = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_laplacian = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_l1 = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_grouplasso = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_laplacian_load = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_l1_load = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_grouplasso_load = np.zeros((len(sample_size),1))
# cum_errWeight_TVFGL_max = np.zeros((len(sample_size),1))
# cum_errWeight_TVFGL_load = np.zeros((len(sample_size),1))
# cum_errWeight_TVFGL_load_zero = np.zeros((len(sample_size),1))

cum_errTheta_GL = np.zeros((len(sample_size),1))
cum_errTheta_FGL = np.zeros((len(sample_size),1))
cum_errTheta_EW = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_laplacian = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_l1 = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_grouplasso = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_laplacian_load = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_l1_load = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_grouplasso_load = np.zeros((len(sample_size),1))
# cum_errTheta_TVFGL_max = np.zeros((len(sample_size),1))
# cum_errTheta_TVFGL_load = np.zeros((len(sample_size),1))
# cum_errTheta_TVFGL_load_zero = np.zeros((len(sample_size),1))

mc_errWeight_GL = np.zeros((ite,1))
mc_errWeight_FGL = np.zeros((ite,1))
mc_errWeight_EW = np.zeros((ite,1))
mc_errWeight_TVFGL_laplacian = np.zeros((ite,1))
mc_errWeight_TVFGL_l1 = np.zeros((ite,1))
mc_errWeight_TVFGL_grouplasso = np.zeros((ite,1))
mc_errWeight_TVFGL_laplacian_load = np.zeros((ite,1))
mc_errWeight_TVFGL_l1_load = np.zeros((ite,1))
mc_errWeight_TVFGL_grouplasso_load = np.zeros((ite,1))
# mc_errWeight_TVFGL_max = np.zeros((ite,1))
# mc_errWeight_TVFGL_load = np.zeros((ite,1))
# mc_errWeight_TVFGL_load_zero = np.zeros((ite,1))

mc_errTheta_GL = np.zeros((ite,1))
mc_errTheta_FGL = np.zeros((ite,1))
mc_errTheta_EW = np.zeros((ite,1))
mc_errTheta_TVFGL_laplacian = np.zeros((ite,1))
mc_errTheta_TVFGL_l1 = np.zeros((ite,1))
mc_errTheta_TVFGL_grouplasso = np.zeros((ite,1))
mc_errTheta_TVFGL_laplacian_load = np.zeros((ite,1))
mc_errTheta_TVFGL_l1_load = np.zeros((ite,1))
mc_errTheta_TVFGL_grouplasso_load = np.zeros((ite,1))
# mc_errTheta_TVFGL_max = np.zeros((ite,1))
# mc_errTheta_TVFGL_load = np.zeros((ite,1))
# mc_errTheta_TVFGL_load_zero = np.zeros((ite,1))
#####################################
gamma_n = np.zeros((len(sample_size),1))
gamma_reps = np.zeros((ite,1))

In [ ]:
#####Appendix Figure E1: TIME-VARYING PRECISION + LOADINGS AND DIFFERENT PENALTY FUNCTIONS
count=-1
for n in sample_size:
    count = count + 1
    p = int(round(n**(0.85),0) )
    #p=round((n[i])^(1.05))# HIGH DIMENSIONAL
    # p=round(3*(n[i])^(0.85))# HIGH DIMENSIONAL
    kDGP = math.ceil(2*(math.log2(p))**(1/2))
    k=kDGP
    kpoet = round(kDGP,0) 
    for jj in range(ite):
        print('n =', n,'jj =', jj)
######################################
    ##########Dynamic Factor DGP##########
    ######################################   
    ###creating Toeplitz matrix and its Cholesky decomposition        
        #######GENERATING LOADINGS##############################
        ####need to create two Lambda matrices associated with different rhos
        ###BEFORE BREAK 1
        rho1 = 0.2
        O1 = np.zeros((p, p))
        np.fill_diagonal(O1, np.ones(p))     
        for h in range(1,p):
            np.fill_diagonal(O1[h:], rho1**np.repeat(h, p-h))
            np.fill_diagonal(O1[:,h:], rho1**np.repeat(h, p-h))
        
        Lambda1 = cholesky(O1, lower=False)
        
        ###AFTER BREAK 1
        rho2 = 0.6
        O2 = np.zeros((p, p))
        np.fill_diagonal(O2, np.ones(p))     
        for h in range(1,p):
            np.fill_diagonal(O2[h:], rho2**np.repeat(h, p-h))
            np.fill_diagonal(O2[:,h:], rho2**np.repeat(h, p-h))
        
        Lambda2 = cholesky(O2, lower=False)
    
        ####AFTER THE REVISION: ALTERNATIVE WAY OF GENERATING FACTORS
        n1 = int(round(n/2))
        n2 = n - n1
        F1 = np.zeros((n1,kDGP))
        F1[0,] = sigmaF*np.random.randn(1,kDGP)
        v=np.random.randn(n1,kDGP)
        for j in range(0,kDGP):
            for t in range(1,n1):
                F1[t,j] = phi*F1[t-1,j]+sigmaF*v[t,j]
                
                
        F2 = np.zeros((n2,kDGP))
        F2[0,] = sigmaF*np.random.randn(1,kDGP)
        v=np.random.randn(n2,kDGP)
        for j in range(0,kDGP):
            for t in range(1,n2):
                F2[t,j] = phi*F2[t-1,j]+sigmaF*v[t,j]        
        
        F = np.concatenate((F1, F2), axis=0)
        covf = np.cov(F, y=None, rowvar=False) #If rowvar is True (default), then each row represents a variable, with observations in the columns                     
        
        
        #################GENERATE EPS##################
        probability = 1- 500/((n)**(0.7)*(p)) #The probability that a coefficient is zero
        ##precision for regime 1
        Theta_u1 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.4, random_state=None)
        Sigma_u1 = inv(Theta_u1)
        ##precision for regime 2
        Theta_u2 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.6, random_state=None)
        Sigma_u2 = inv(Theta_u2)

###########AFTER THE REVISION###############                
        eps1 = np.zeros((n1,p))
        eps2 = np.zeros((n2,p))
        breaks = np.zeros((n,1))
        mue = np.zeros((p,1))
        for jjj in range(0,n):
            if jjj <= n1:
                breaks[jjj,]=0
            else:
                breaks[jjj,]=1
        for jjj in range(0,n1):
            eps1[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u1,1)
        for jjj in range(0,n2):
            eps2[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u1,1)
      
        breaks = breaks.astype(int)
        breaks = np.ravel(breaks)
        eps = np.concatenate((eps1, eps2), axis=0)
        cov_u1 = np.cov(eps1, y=None, rowvar=False) 
        Theta_u1 = np.linalg.inv(cov_u1)
        
        cov_u2 = np.cov(eps2, y=None, rowvar=False) 
        Theta_u2 = np.linalg.inv(cov_u2)
        
        #########AFTER THE REVISION
        r1 = np.zeros((n1,p))
        r2 = np.zeros((n2,p))
        
        r1 = F1@Lambda1[0:kDGP,] + eps1
        r2 = F2@Lambda2[0:kDGP,] + eps2
        
        r = np.concatenate((r1, r2), axis=0)
        ###################################
        #####AFTER THE REVISION####
        icov = np.cov(r, y=None, rowvar=False) 
        isig = np.linalg.inv(icov)
        
        
      ###Incorporating info about time-varying factor loadings
 
      ###############################################################
        ##############Estimating factors and loadings for time-varying########
        gamma_opt = CV_gamma(gamma_set,r, r1, r2, k)
        print('gamma_opt =', gamma_opt)
        gamma_reps[jj] = gamma_opt
        ####modified returns for time-varying loadings only!!!
        r_load = r.copy()
        for row in range(r_load.shape[0]):
            for col in range(r_load.shape[1]):
                if row <= n1:
                    r_load[row,col]=gamma_opt* r_load[row,col] 

        L_load, V_load = np.linalg.eigh(np.dot(r_load.T, r_load))
        idx_load = L_load.argsort()[::-1]
        L_load = L_load[idx_load]  # eigenvalues, Nx1
        V_load = V_load[:, idx_load]  # eigenvectors columns, NxN
        lmb_load = V_load[:, 0:k]  # kx1
        # Fhat = np.dot(r, lmb)  # Txr (r=1 for PC1)
        ###According to Su (2017, JoE) if we obtain Fhat
        ###as usual they are only consistent for a rotational version
        ###hence, to get a consistent estimator use a two-stage procedure (OLS)
        Fhat_load = r@lmb_load@np.linalg.inv(lmb_load.T@lmb_load)
        Y_load = r - Fhat_load@lmb_load.T ##these are the residuals
        
        covariate_load = Fhat_load
        betas_load = lmb_load.T
        
        
        ##############Estimating USUAL factors and loadings (gamma=1)########
        L, V = np.linalg.eigh(np.dot(r.T, r))
        idx = L.argsort()[::-1]
        L = L[idx]  # eigenvalues, Nx1
        V = V[:, idx]  # eigenvectors columns, NxN
        lmb = V[:, 0:k]  # kx1
        Fhat = np.dot(r, lmb)  # Txr (r=1 for PC1)
        Y = r - Fhat@lmb.T ##these are the residuals
        
        covariate = Fhat
        betas = lmb.T

        ############Estimating precision matrix and combination weights using competing methods######
        ######## GLASSO###########
        GL = GraphicalLassoCV().fit(r)
        theta_GL = GL.get_precision()
        
        ########Factor GLASSO#######
        FGL = GraphicalLassoCV().fit(Y)
        theta_FGL_error = FGL.get_precision()
        
        if k==1:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
        else:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            
        ########Time-Varying TVFGL (gamma=1)#########
        denominator = p
        
        tuning_laplacian = CV_theta(penalty = '',Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)##laplacian (ridge) is defauls and was used at the beginning
        tuning_l1 = CV_theta(penalty = 'l1', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        tuning_grouplasso = CV_theta(penalty = 'l2', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
#         tuning_max = CV_theta(penalty = 'linf', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
      
        print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)
        ###laplacian 
        tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############l1
        tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############group lasso
        tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
                
         ########Time-Varying TVFGL(BOTH precision and loadings time-varying)#########        
        tuning_laplacian_load = CV_theta(penalty = '',Y=Y_load,breaks = breaks, betas = betas_load, covariate=covariate_load, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)##laplacian (ridge) is defauls and was used at the beginning
        tuning_l1_load = CV_theta(penalty = 'l1', Y=Y_load,breaks = breaks, betas = betas_load, covariate=covariate_load, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        tuning_grouplasso_load = CV_theta(penalty = 'l2', Y=Y_load,breaks = breaks, betas = betas_load, covariate=covariate_load, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
#         tuning_max = CV_theta(penalty = 'linf', Y=Y_load,breaks = breaks, betas = betas_load, covariate=covariate_load, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
      
        print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)

        ###laplacian       
        tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y_load, breaks)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
        if k==1:
            theta_TVFGL_laplacian_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_laplacian_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
     
        ###l1       
        tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y_load, breaks)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
        if k==1:
            theta_TVFGL_l1_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_l1_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
     ###############################################################   
        ###group lasso     
        tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y_load, breaks)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
        if k==1:
            theta_TVFGL_grouplasso_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_grouplasso_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
     ###############################################################   
        
        
        
        #################EQUALLY WEIGHTED########################
        ev = np.linalg.eig(np.cov(r, y=None, rowvar=False))
        eigenvalues = ev[0]
        mu=np.mean(eigenvalues)   
        CovshrIdent = np.zeros((p, p))
        np.fill_diagonal(CovshrIdent, np.repeat(mu, p)) 
        theta_EW=np.linalg.inv(CovshrIdent)
        
        
        ##################################################
        ######Estimation errors for Precision Matrix######
        ##################################################   
        errTheta_GL = math.log2(np.linalg.norm(theta_GL-isig, ord=2)/np.sqrt(denominator))
        errTheta_FGL = math.log2(np.linalg.norm(theta_FGL-isig, ord=2)/np.sqrt(denominator))
        errTheta_EW = math.log2(np.linalg.norm(theta_EW-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_laplacian = math.log2(np.linalg.norm(theta_TVFGL_laplacian-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_l1 = math.log2(np.linalg.norm(theta_TVFGL_l1-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_grouplasso = math.log2(np.linalg.norm(theta_TVFGL_grouplasso-isig, ord=2)/np.sqrt(denominator))
#         errTheta_TVFGL_max = math.log2(np.linalg.norm(theta_TVFGL_max-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_laplacian_load = math.log2(np.linalg.norm(theta_TVFGL_laplacian_load-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_l1_load = math.log2(np.linalg.norm(theta_TVFGL_l1_load-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_grouplasso_load = math.log2(np.linalg.norm(theta_TVFGL_grouplasso_load-isig, ord=2)/np.sqrt(denominator))
        ##################################################
        ######Estimation errors for Portfolio Weights######
        ##################################################
        weight_true = portfolios(r,isig)[0]
        
        weight_GL = portfolios(r,theta_GL)[0]
        weight_FGL = portfolios(r,theta_FGL)[0]
        weight_EW = portfolios(r,theta_EW)[0]
        weight_TVFGL_laplacian = portfolios(r,theta_TVFGL_laplacian)[0]
        weight_TVFGL_l1 = portfolios(r,theta_TVFGL_l1)[0]
        weight_TVFGL_grouplasso = portfolios(r,theta_TVFGL_grouplasso)[0]
        weight_TVFGL_laplacian_load = portfolios(r,theta_TVFGL_laplacian_load)[0]
        weight_TVFGL_l1_load = portfolios(r,theta_TVFGL_l1_load)[0]
        weight_TVFGL_grouplasso_load = portfolios(r,theta_TVFGL_grouplasso_load)[0]
        ############Global Minimum Variance (GMV)#########  
        errWeight_GL = math.log2(np.linalg.norm(weight_GL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_FGL = math.log2(np.linalg.norm(weight_FGL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_EW = math.log2(np.linalg.norm(weight_EW-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_laplacian = math.log2(np.linalg.norm(weight_TVFGL_laplacian-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_l1 = math.log2(np.linalg.norm(weight_TVFGL_l1-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_grouplasso = math.log2(np.linalg.norm(weight_TVFGL_grouplasso-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_laplacian_load = math.log2(np.linalg.norm(weight_TVFGL_laplacian_load-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_l1_load = math.log2(np.linalg.norm(weight_TVFGL_l1_load-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_grouplasso_load = math.log2(np.linalg.norm(weight_TVFGL_grouplasso_load-weight_true, ord=1)/np.sqrt(denominator))
        
    ############Precision Matrix#########  
        mc_errTheta_GL[jj] = errTheta_GL    
        mc_errTheta_FGL[jj] = errTheta_FGL
        mc_errTheta_EW[jj] = errTheta_EW
        mc_errTheta_TVFGL_laplacian[jj] = errTheta_TVFGL_laplacian
        mc_errTheta_TVFGL_l1[jj] = errTheta_TVFGL_l1
        mc_errTheta_TVFGL_grouplasso[jj] = errTheta_TVFGL_grouplasso
        mc_errTheta_TVFGL_laplacian_load[jj] = errTheta_TVFGL_laplacian_load
        mc_errTheta_TVFGL_l1_load[jj] = errTheta_TVFGL_l1_load
        mc_errTheta_TVFGL_grouplasso_load[jj] = errTheta_TVFGL_grouplasso_load
        
    ############Global Minimum Variance (GMV)#########   
        mc_errWeight_GL[jj] = errWeight_GL    
        mc_errWeight_FGL[jj] = errWeight_FGL
        mc_errWeight_EW[jj] = errWeight_EW
        mc_errWeight_TVFGL_laplacian[jj] = errWeight_TVFGL_laplacian
        mc_errWeight_TVFGL_l1[jj] = errWeight_TVFGL_l1
        mc_errWeight_TVFGL_grouplasso[jj] = errWeight_TVFGL_grouplasso
        mc_errWeight_TVFGL_laplacian_load[jj] = errWeight_TVFGL_laplacian_load
        mc_errWeight_TVFGL_l1_load[jj] = errWeight_TVFGL_l1_load
        mc_errWeight_TVFGL_grouplasso_load[jj] = errWeight_TVFGL_grouplasso_load
  ############Precision Matrix#########
    gamma_n[count] = np.mean(gamma_reps)
    
    cum_errTheta_GL[count] = np.mean(mc_errTheta_GL)
    cum_errTheta_FGL[count] = np.mean(mc_errTheta_FGL)
    cum_errTheta_EW[count] = np.mean(mc_errTheta_EW)
    cum_errTheta_TVFGL_laplacian[count] = np.mean(mc_errTheta_TVFGL_laplacian)
    cum_errTheta_TVFGL_l1[count] = np.mean(mc_errTheta_TVFGL_l1)
    cum_errTheta_TVFGL_grouplasso[count] = np.mean(mc_errTheta_TVFGL_grouplasso)
    cum_errTheta_TVFGL_laplacian_load[count] = np.mean(mc_errTheta_TVFGL_laplacian_load)
    cum_errTheta_TVFGL_l1_load[count] = np.mean(mc_errTheta_TVFGL_l1_load)
    cum_errTheta_TVFGL_grouplasso_load[count] = np.mean(mc_errTheta_TVFGL_grouplasso_load)
    
    ############Global Minimum Variance (GMV)#########
    cum_errWeight_GL[count] = np.mean(mc_errWeight_GL)
    cum_errWeight_FGL[count] = np.mean(mc_errWeight_FGL)
    cum_errWeight_EW[count] = np.mean(mc_errWeight_EW)
    cum_errWeight_TVFGL_laplacian[count] = np.mean(mc_errWeight_TVFGL_laplacian)
    cum_errWeight_TVFGL_l1[count] = np.mean(mc_errWeight_TVFGL_l1)
    cum_errWeight_TVFGL_grouplasso[count] = np.mean(mc_errWeight_TVFGL_grouplasso)
    cum_errWeight_TVFGL_laplacian_load[count] = np.mean(mc_errWeight_TVFGL_laplacian_load)
    cum_errWeight_TVFGL_l1_load[count] = np.mean(mc_errWeight_TVFGL_l1_load)
    cum_errWeight_TVFGL_grouplasso_load[count] = np.mean(mc_errWeight_TVFGL_grouplasso_load)
    
cum_errTheta = np.concatenate((cum_errTheta_FGL,cum_errTheta_GL,cum_errTheta_EW,cum_errTheta_TVFGL_laplacian,cum_errTheta_TVFGL_l1,cum_errTheta_TVFGL_grouplasso,cum_errTheta_TVFGL_laplacian_load,cum_errTheta_TVFGL_l1_load,cum_errTheta_TVFGL_grouplasso_load), axis=1)
savetxt('cum_errTheta_break_theta.csv', cum_errTheta, delimiter=',')

cum_errWeight = np.concatenate((cum_errWeight_FGL,cum_errWeight_GL,cum_errWeight_EW,cum_errWeight_TVFGL_laplacian,cum_errWeight_TVFGL_l1,cum_errWeight_TVFGL_grouplasso,cum_errWeight_TVFGL_laplacian_load,cum_errWeight_TVFGL_l1_load,cum_errWeight_TVFGL_grouplasso_load), axis=1)
savetxt('cum_errWeight_break_theta_and_loadings.csv', cum_errWeight, delimiter=',')

savetxt('gamma_break_theta_and_loadings.csv', gamma_n, delimiter=',')
      
 

In [ ]:
#####Appendix Figure E3: simulations without any break
count=-1
for n in sample_size:
    count = count + 1
    p = int(round(n**(0.85),0) )
    #p=round((n[i])^(1.05))# HIGH DIMENSIONAL
    # p=round(3*(n[i])^(0.85))# HIGH DIMENSIONAL
    kDGP = math.ceil(2*(math.log2(p))**(1/2))
    k=kDGP
    kpoet = round(kDGP,0) 
    for jj in range(ite):
        print('n =', n,'jj =', jj, 'out of =', ite)
    ######################################
    ##########Dynamic Factor DGP##########
    ######################################   
    ###creating Toeplitz matrix and its Cholesky decomposition
    ###WITHOUT BREAK
        O = np.zeros((p, p))
        np.fill_diagonal(O, np.ones(p))     
        for h in range(1,p):
            np.fill_diagonal(O[h:], rho**np.repeat(h, p-h))
            np.fill_diagonal(O[:,h:], rho**np.repeat(h, p-h))
        
        Lambda = cholesky(O, lower=False)
        
    
        #######BEFORE THE REVISION: GENERATING FACTORS###############################
        F = np.zeros((n,kDGP))
        F[0,] = sigmaF*np.random.randn(1,kDGP)
        v=np.random.randn(n,kDGP)
        for j in range(0,kDGP):
            for t in range(1,n):
                F[t,j] = phi*F[t-1,j]+sigmaF*v[t,j]
        
        covf = np.cov(F, y=None, rowvar=False) #If rowvar is True (default), then each row represents a variable, with observations in the columns                     
        
        
        #################GENERATE EPS##################
        probability = 1- 500/((n)**(0.7)*(p)) #The probability that a coefficient is zero
        ##precision for regime 1
        Theta_u = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.3, random_state=None)
        Sigma_u = inv(Theta_u)
        
#########BEFORE THE REVISION################
        eps = np.zeros((n,p))
        breaks = np.zeros((n,1))
        mue = np.zeros((p,1))
        for jjj in range(0,n):
            eps[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u,1)
        breaks = breaks.astype(int)
        breaks = np.ravel(breaks)
        cov_u = np.cov(eps, y=None, rowvar=False) 
        Theta_u = np.linalg.inv(cov_u)
##################################################
    
        
        #########AFTER THE REVISION
        r = np.zeros((n,p))        
        r = F@Lambda[0:kDGP,] + eps
        icov = np.cov(r, y=None, rowvar=False) 
        isig = np.linalg.inv(icov)

        ###############################################################################
        
        ##############Estimating USUAL factors and loadings (gamma=1)########
        L, V = np.linalg.eigh(np.dot(r.T, r))
        idx = L.argsort()[::-1]
        L = L[idx]  # eigenvalues, Nx1
        V = V[:, idx]  # eigenvectors columns, NxN
        lmb = V[:, 0:k]  # kx1
        Fhat = np.dot(r, lmb)  # Txr (r=1 for PC1)
        Y = r - Fhat@lmb.T ##these are the residuals
        
        covariate = Fhat
        betas = lmb.T

        ############Estimating precision matrix and combination weights using competing methods######
        ######## GLASSO###########
        GL = GraphicalLassoCV().fit(r)
        theta_GL = GL.get_precision()
        
        ########Factor GLASSO#######
        FGL = GraphicalLassoCV().fit(Y)
        theta_FGL_error = FGL.get_precision()
        
        if k==1:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
        else:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            
        ########Time-Varying TVFGL (gamma=1)#########
        denominator = p
        
        tuning_laplacian = CV_theta(penalty = '',Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)##laplacian (ridge) is defauls and was used at the beginning
        tuning_l1 = CV_theta(penalty = 'l1', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        tuning_grouplasso = CV_theta(penalty = 'l2', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        tuning_max = CV_theta(penalty = 'linf', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
      
        
        
#         tuning = CV_theta(Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        print(tuning_laplacian.T)
        ########laplacian
        tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############l1
        tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############group lasso
        tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############max norm
        tvfgl = TimeGraphicalLasso(psi = 'linf', max_iter=100, alpha = tuning_max[0][0], beta = tuning_max[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #################EQUALLY WEIGHTED########################
        ev = np.linalg.eig(np.cov(r, y=None, rowvar=False))
        eigenvalues = ev[0]
        mu=np.mean(eigenvalues)   
        CovshrIdent = np.zeros((p, p))
        np.fill_diagonal(CovshrIdent, np.repeat(mu, p)) 
        theta_EW=np.linalg.inv(CovshrIdent)
        
        ##################################################
        ######Estimation errors for Precision Matrix######
        ##################################################   
        errTheta_GL = math.log2(np.linalg.norm(theta_GL-isig, ord=2)/np.sqrt(denominator))
        errTheta_FGL = math.log2(np.linalg.norm(theta_FGL-isig, ord=2)/np.sqrt(denominator))
        errTheta_EW = math.log2(np.linalg.norm(theta_EW-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_laplacian = math.log2(np.linalg.norm(theta_TVFGL_laplacian-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_l1 = math.log2(np.linalg.norm(theta_TVFGL_l1-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_grouplasso = math.log2(np.linalg.norm(theta_TVFGL_grouplasso-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_max = math.log2(np.linalg.norm(theta_TVFGL_max-isig, ord=2)/np.sqrt(denominator))
        
#         errTheta_TVFGL_load = math.log2(np.linalg.norm(theta_TVFGL_load-isig, ord=2)/np.sqrt(denominator))
#         errTheta_TVFGL_load_zero = math.log2(np.linalg.norm(theta_TVFGL_load_zero-isig, ord=2)/np.sqrt(denominator))
        
        ##################################################
        ######Estimation errors for Portfolio Weights######
        ##################################################
        weight_true = portfolios(r,isig)[0]
        
        weight_GL = portfolios(r,theta_GL)[0]
        weight_FGL = portfolios(r,theta_FGL)[0]
        weight_EW = portfolios(r,theta_EW)[0]
        weight_TVFGL_laplacian = portfolios(r,theta_TVFGL_laplacian)[0]
        weight_TVFGL_l1 = portfolios(r,theta_TVFGL_l1)[0]
        weight_TVFGL_grouplasso = portfolios(r,theta_TVFGL_grouplasso)[0]
        weight_TVFGL_max = portfolios(r,theta_TVFGL_max)[0]
#         weight_TVFGL_load = portfolios(r,theta_TVFGL_load)[0]
#         weight_TVFGL_load_zero = portfolios(r,theta_TVFGL_load_zero)[0]
        ############Global Minimum Variance (GMV)#########  
        errWeight_GL = math.log2(np.linalg.norm(weight_GL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_FGL = math.log2(np.linalg.norm(weight_FGL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_EW = math.log2(np.linalg.norm(weight_EW-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_laplacian = math.log2(np.linalg.norm(weight_TVFGL_laplacian-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_load = math.log2(np.linalg.norm(weight_TVFGL_load-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_load_zero = math.log2(np.linalg.norm(weight_TVFGL_load_zero-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_l1 = math.log2(np.linalg.norm(weight_TVFGL_l1-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_grouplasso = math.log2(np.linalg.norm(weight_TVFGL_grouplasso-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_max = math.log2(np.linalg.norm(weight_TVFGL_max-weight_true, ord=1)/np.sqrt(denominator))
    ############Precision Matrix#########  
        mc_errTheta_GL[jj] = errTheta_GL    
        mc_errTheta_FGL[jj] = errTheta_FGL
        mc_errTheta_EW[jj] = errTheta_EW
        mc_errTheta_TVFGL_laplacian[jj] = errTheta_TVFGL_laplacian
        mc_errTheta_TVFGL_l1[jj] = errTheta_TVFGL_l1
        mc_errTheta_TVFGL_grouplasso[jj] = errTheta_TVFGL_grouplasso
        mc_errTheta_TVFGL_max[jj] = errTheta_TVFGL_max
#         mc_errTheta_TVFGL_load[jj] = errTheta_TVFGL_load
#         mc_errTheta_TVFGL_load_zero[jj] = errTheta_TVFGL_load_zero
        
    ############Global Minimum Variance (GMV)#########   
        mc_errWeight_GL[jj] = errWeight_GL    
        mc_errWeight_FGL[jj] = errWeight_FGL
        mc_errWeight_EW[jj] = errWeight_EW
        mc_errWeight_TVFGL_laplacian[jj] = errWeight_TVFGL_laplacian
        mc_errWeight_TVFGL_l1[jj] = errWeight_TVFGL_l1
        mc_errWeight_TVFGL_grouplasso[jj] = errWeight_TVFGL_grouplasso
        mc_errWeight_TVFGL_max[jj] = errWeight_TVFGL_max
#         mc_errWeight_TVFGL_load[jj] = errWeight_TVFGL_load
#         mc_errWeight_TVFGL_load_zero[jj] = errWeight_TVFGL_load_zero
  ############Precision Matrix#########
    gamma_n[count] = np.mean(gamma_reps)
    
    cum_errTheta_GL[count] = np.mean(mc_errTheta_GL)
    cum_errTheta_FGL[count] = np.mean(mc_errTheta_FGL)
    cum_errTheta_EW[count] = np.mean(mc_errTheta_EW)
    cum_errTheta_TVFGL_laplacian[count] = np.mean(mc_errTheta_TVFGL_laplacian)
    cum_errTheta_TVFGL_l1[count] = np.mean(mc_errTheta_TVFGL_l1)
    cum_errTheta_TVFGL_grouplasso[count] = np.mean(mc_errTheta_TVFGL_grouplasso)
    cum_errTheta_TVFGL_max[count] = np.mean(mc_errTheta_TVFGL_max)
#     cum_errTheta_TVFGL_load[count] = np.mean(mc_errTheta_TVFGL_load)
#     cum_errTheta_TVFGL_load_zero[count] = np.mean(mc_errTheta_TVFGL_load_zero)
    
    ############Global Minimum Variance (GMV)#########
    cum_errWeight_GL[count] = np.mean(mc_errWeight_GL)
    cum_errWeight_FGL[count] = np.mean(mc_errWeight_FGL)
    cum_errWeight_EW[count] = np.mean(mc_errWeight_EW)
    cum_errWeight_TVFGL_laplacian[count] = np.mean(mc_errWeight_TVFGL_laplacian)
    cum_errWeight_TVFGL_l1[count] = np.mean(mc_errWeight_TVFGL_l1)
    cum_errWeight_TVFGL_grouplasso[count] = np.mean(mc_errWeight_TVFGL_grouplasso)
    cum_errWeight_TVFGL_max[count] = np.mean(mc_errWeight_TVFGL_max)
#     cum_errWeight_TVFGL_load[count] = np.mean(mc_errWeight_TVFGL_load)
#     cum_errWeight_TVFGL_load_zero[count] = np.mean(mc_errWeight_TVFGL_load_zero)
    
cum_errTheta = np.concatenate((cum_errTheta_FGL,cum_errTheta_GL,cum_errTheta_EW,cum_errTheta_TVFGL_laplacian,cum_errTheta_TVFGL_l1,cum_errTheta_TVFGL_grouplasso,cum_errTheta_TVFGL_max), axis=1)
savetxt('cum_errTheta_nobreak.csv', cum_errTheta, delimiter=',')

cum_errWeight = np.concatenate((cum_errWeight_FGL,cum_errWeight_GL,cum_errWeight_EW,cum_errWeight_TVFGL_laplacian,cum_errWeight_TVFGL_l1,cum_errWeight_TVFGL_grouplasso,cum_errWeight_TVFGL_max), axis=1)
savetxt('cum_errWeight_nobreak.csv', cum_errWeight, delimiter=',')

savetxt('gamma_nobreak.csv', gamma_n, delimiter=',')

In [ ]:
#####Appendix Figure E5: TIME-VARYING PRECISION (fixed loadings) AND DIFFERENT PENALTY FUNCTIONS
count=-1
for n in sample_size:
    count = count + 1
    p = int(round(n**(0.85),0) )
    #p=round((n[i])^(1.05))# HIGH DIMENSIONAL
    # p=round(3*(n[i])^(0.85))# HIGH DIMENSIONAL
    kDGP = math.ceil(2*(math.log2(p))**(1/2))
    k=kDGP
    kpoet = round(kDGP,0) 
    for jj in range(ite):
        print('n =', n,'jj =', jj)
    ######################################
    ##########Dynamic Factor DGP##########
    ######################################   
    ###creating Toeplitz matrix and its Cholesky decomposition
    ###WITHOUT BREAK
        O = np.zeros((p, p))
        np.fill_diagonal(O, np.ones(p))     
        for h in range(1,p):
            np.fill_diagonal(O[h:], rho**np.repeat(h, p-h))
            np.fill_diagonal(O[:,h:], rho**np.repeat(h, p-h))
        
        Lambda = cholesky(O, lower=False)
        
    
        #######BEFORE THE REVISION: GENERATING FACTORS###############################
        F = np.zeros((n,kDGP))
        F[0,] = sigmaF*np.random.randn(1,kDGP)
        v=np.random.randn(n,kDGP)
        for j in range(0,kDGP):
            for t in range(1,n):
                F[t,j] = phi*F[t-1,j]+sigmaF*v[t,j]
        
        covf = np.cov(F, y=None, rowvar=False) #If rowvar is True (default), then each row represents a variable, with observations in the columns                     
        
        n1 = int(round(n/2))
        n2 = n - n1
        #################GENERATE EPS##################
        probability = 1- 500/((n)**(0.7)*(p)) #The probability that a coefficient is zero
        ##precision for regime 1
        Theta_u1 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.4, random_state=None)
        Sigma_u1 = inv(Theta_u1)
        ##precision for regime 2
        Theta_u2 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.6, random_state=None)
        Sigma_u2 = inv(Theta_u2)
        

###########AFTER THE REVISION###############                
        eps1 = np.zeros((n1,p))
        eps2 = np.zeros((n2,p))
        breaks = np.zeros((n,1))
        mue = np.zeros((p,1))
        for jjj in range(0,n):
            if jjj <= n1:
                breaks[jjj,]=0
            else:
                breaks[jjj,]=1
        for jjj in range(0,n1):
            eps1[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u1,1)
        for jjj in range(0,n2):
            eps2[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u2,1)
      
        breaks = breaks.astype(int)
        breaks = np.ravel(breaks)
        eps = np.concatenate((eps1, eps2), axis=0)
        cov_u1 = np.cov(eps1, y=None, rowvar=False) 
        Theta_u1 = np.linalg.inv(cov_u1)
        
        cov_u2 = np.cov(eps2, y=None, rowvar=False) 
        Theta_u2 = np.linalg.inv(cov_u2)
        
    
        #################GENERATE RETURNS#################
        
        #########AFTER THE REVISION
        r = np.zeros((n,p))        
        r = F@Lambda[0:kDGP,] + eps

        ###################################
        ######BEFORE THE REVISION
        # r = F@Lambda[0:kDGP,] + eps
        # isig = Theta_u - Theta_u@Lambda[0:kDGP,].T@np.linalg.inv( np.linalg.inv(covf)+Lambda[0:kDGP,]@Theta_u@Lambda[0:kDGP,].T)@Lambda[0:kDGP,]@Theta_u
        
        
        #####AFTER THE REVISION####
        ##ON SECOND THOUGHT: how about just taking the inverse of returns?
        icov = np.cov(r, y=None, rowvar=False) 
        isig = np.linalg.inv(icov)
        ###two different population precision matrices before and after the break
        # isig1 = Theta_u1 - Theta_u1@Lambda1[0:kDGP,].T@np.linalg.inv( np.linalg.inv(covf)+Lambda1[0:kDGP,]@Theta_u1@Lambda1[0:kDGP,].T)@Lambda1[0:kDGP,]@Theta_u1
        # isig2 = Theta_u2 - Theta_u2@Lambda2[0:kDGP,].T@np.linalg.inv( np.linalg.inv(covf)+Lambda2[0:kDGP,]@Theta_u2@Lambda2[0:kDGP,].T)@Lambda2[0:kDGP,]@Theta_u2
        
      ###############################################################
       
        
        ##############Estimating USUAL factors and loadings (gamma=1)########
        L, V = np.linalg.eigh(np.dot(r.T, r))
        idx = L.argsort()[::-1]
        L = L[idx]  # eigenvalues, Nx1
        V = V[:, idx]  # eigenvectors columns, NxN
        lmb = V[:, 0:k]  # kx1
        Fhat = np.dot(r, lmb)  # Txr (r=1 for PC1)
        Y = r - Fhat@lmb.T ##these are the residuals
        
        covariate = Fhat
        betas = lmb.T

        ############Estimating precision matrix and combination weights using competing methods######
        ######## GLASSO###########
        GL = GraphicalLassoCV().fit(r)
        theta_GL = GL.get_precision()
        
        ########Factor GLASSO#######
        FGL = GraphicalLassoCV().fit(Y)
        theta_FGL_error = FGL.get_precision()
        
        if k==1:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
        else:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            
        ########Time-Varying TVFGL (gamma=1)#########
        ########Time-Varying TVFGL (gamma=1)#########
        denominator = p
        
        tuning_laplacian = CV_theta(penalty = '',Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)##laplacian (ridge) is defauls and was used at the beginning
        tuning_l1 = CV_theta(penalty = 'l1', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        tuning_grouplasso = CV_theta(penalty = 'l2', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
#         tuning_max = CV_theta(penalty = 'linf', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
      
        
        
#         tuning = CV_theta(Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)
        ###laplacian 
        tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############l1
        tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############group lasso
        tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
#         #############max norm
#         tvfgl = TimeGraphicalLasso(psi = 'linf', max_iter=100, alpha = tuning_max[0][0], beta = tuning_max[1][0]).fit(Y, breaks)
#         if k==1:
#             theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
#         else:
#             theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #################EQUALLY WEIGHTED########################
        ev = np.linalg.eig(np.cov(r, y=None, rowvar=False))
        eigenvalues = ev[0]
        mu=np.mean(eigenvalues)   
        CovshrIdent = np.zeros((p, p))
        np.fill_diagonal(CovshrIdent, np.repeat(mu, p)) 
        theta_EW=np.linalg.inv(CovshrIdent)
        
        
        ##################################################
        ######Estimation errors for Precision Matrix######
        ##################################################   
        errTheta_GL = math.log2(np.linalg.norm(theta_GL-isig, ord=2)/np.sqrt(denominator))
        errTheta_FGL = math.log2(np.linalg.norm(theta_FGL-isig, ord=2)/np.sqrt(denominator))
        errTheta_EW = math.log2(np.linalg.norm(theta_EW-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_laplacian = math.log2(np.linalg.norm(theta_TVFGL_laplacian-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_l1 = math.log2(np.linalg.norm(theta_TVFGL_l1-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_grouplasso = math.log2(np.linalg.norm(theta_TVFGL_grouplasso-isig, ord=2)/np.sqrt(denominator))
#         errTheta_TVFGL_max = math.log2(np.linalg.norm(theta_TVFGL_max-isig, ord=2)/np.sqrt(denominator))
        
        ##################################################
        ######Estimation errors for Portfolio Weights######
        ##################################################
        weight_true = portfolios(r,isig)[0]
        
        weight_GL = portfolios(r,theta_GL)[0]
        weight_FGL = portfolios(r,theta_FGL)[0]
        weight_EW = portfolios(r,theta_EW)[0]
        weight_TVFGL_laplacian = portfolios(r,theta_TVFGL_laplacian)[0]
        weight_TVFGL_l1 = portfolios(r,theta_TVFGL_l1)[0]
        weight_TVFGL_grouplasso = portfolios(r,theta_TVFGL_grouplasso)[0]
#         weight_TVFGL_max = portfolios(r,theta_TVFGL_max)[0]
#         weight_TVFGL_load = portfolios(r,theta_TVFGL_load)[0]
#         weight_TVFGL_load_zero = portfolios(r,theta_TVFGL_load_zero)[0]
        ############Global Minimum Variance (GMV)#########  
        errWeight_GL = math.log2(np.linalg.norm(weight_GL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_FGL = math.log2(np.linalg.norm(weight_FGL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_EW = math.log2(np.linalg.norm(weight_EW-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_laplacian = math.log2(np.linalg.norm(weight_TVFGL_laplacian-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_load = math.log2(np.linalg.norm(weight_TVFGL_load-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_load_zero = math.log2(np.linalg.norm(weight_TVFGL_load_zero-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_l1 = math.log2(np.linalg.norm(weight_TVFGL_l1-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_grouplasso = math.log2(np.linalg.norm(weight_TVFGL_grouplasso-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_max = math.log2(np.linalg.norm(weight_TVFGL_max-weight_true, ord=1)/np.sqrt(denominator))
    ############Precision Matrix#########  
        mc_errTheta_GL[jj] = errTheta_GL    
        mc_errTheta_FGL[jj] = errTheta_FGL
        mc_errTheta_EW[jj] = errTheta_EW
        mc_errTheta_TVFGL_laplacian[jj] = errTheta_TVFGL_laplacian
        mc_errTheta_TVFGL_l1[jj] = errTheta_TVFGL_l1
        mc_errTheta_TVFGL_grouplasso[jj] = errTheta_TVFGL_grouplasso
#         mc_errTheta_TVFGL_max[jj] = errTheta_TVFGL_max
#         mc_errTheta_TVFGL_load[jj] = errTheta_TVFGL_load
#         mc_errTheta_TVFGL_load_zero[jj] = errTheta_TVFGL_load_zero
        
    ############Global Minimum Variance (GMV)#########   
        mc_errWeight_GL[jj] = errWeight_GL    
        mc_errWeight_FGL[jj] = errWeight_FGL
        mc_errWeight_EW[jj] = errWeight_EW
        mc_errWeight_TVFGL_laplacian[jj] = errWeight_TVFGL_laplacian
        mc_errWeight_TVFGL_l1[jj] = errWeight_TVFGL_l1
        mc_errWeight_TVFGL_grouplasso[jj] = errWeight_TVFGL_grouplasso
#         mc_errWeight_TVFGL_max[jj] = errWeight_TVFGL_max
#         mc_errWeight_TVFGL_load[jj] = errWeight_TVFGL_load
#         mc_errWeight_TVFGL_load_zero[jj] = errWeight_TVFGL_load_zero
  ############Precision Matrix#########
    gamma_n[count] = np.mean(gamma_reps)
    
    cum_errTheta_GL[count] = np.mean(mc_errTheta_GL)
    cum_errTheta_FGL[count] = np.mean(mc_errTheta_FGL)
    cum_errTheta_EW[count] = np.mean(mc_errTheta_EW)
    cum_errTheta_TVFGL_laplacian[count] = np.mean(mc_errTheta_TVFGL_laplacian)
    cum_errTheta_TVFGL_l1[count] = np.mean(mc_errTheta_TVFGL_l1)
    cum_errTheta_TVFGL_grouplasso[count] = np.mean(mc_errTheta_TVFGL_grouplasso)
#     cum_errTheta_TVFGL_max[count] = np.mean(mc_errTheta_TVFGL_max)
#     cum_errTheta_TVFGL_load[count] = np.mean(mc_errTheta_TVFGL_load)
#     cum_errTheta_TVFGL_load_zero[count] = np.mean(mc_errTheta_TVFGL_load_zero)
    
    ############Global Minimum Variance (GMV)#########
    cum_errWeight_GL[count] = np.mean(mc_errWeight_GL)
    cum_errWeight_FGL[count] = np.mean(mc_errWeight_FGL)
    cum_errWeight_EW[count] = np.mean(mc_errWeight_EW)
    cum_errWeight_TVFGL_laplacian[count] = np.mean(mc_errWeight_TVFGL_laplacian)
    cum_errWeight_TVFGL_l1[count] = np.mean(mc_errWeight_TVFGL_l1)
    cum_errWeight_TVFGL_grouplasso[count] = np.mean(mc_errWeight_TVFGL_grouplasso)
#     cum_errWeight_TVFGL_max[count] = np.mean(mc_errWeight_TVFGL_max)
#     cum_errWeight_TVFGL_load[count] = np.mean(mc_errWeight_TVFGL_load)
#     cum_errWeight_TVFGL_load_zero[count] = np.mean(mc_errWeight_TVFGL_load_zero)
    
cum_errTheta = np.concatenate((cum_errTheta_FGL,cum_errTheta_GL,cum_errTheta_EW,cum_errTheta_TVFGL_laplacian,cum_errTheta_TVFGL_l1,cum_errTheta_TVFGL_grouplasso), axis=1)
savetxt('cum_errTheta_break_theta.csv', cum_errTheta, delimiter=',')

cum_errWeight = np.concatenate((cum_errWeight_FGL,cum_errWeight_GL,cum_errWeight_EW,cum_errWeight_TVFGL_laplacian,cum_errWeight_TVFGL_l1,cum_errWeight_TVFGL_grouplasso), axis=1)
savetxt('cum_errWeight_break_theta.csv', cum_errWeight, delimiter=',')

savetxt('gamma_break_theta.csv', gamma_n, delimiter=',')
      
 

In [ ]:
########Appendix Figure E6: FIXED LOADINGS AND TWO BREAKS IN IDIOSYNCRATIC PRECISION: TIME-VARYING PRECISION (fixed loadings) AND DIFFERENT PENALTY FUNCTIONS
###INCLUDES THE CASE WITH FIXED LOADINGS AND TWO BREAKS IN IDIOSYNCRATIC PRECISION

###PARAMETERS
gamma_opt_zero = 0
gamma_set = np.arange(0,1.05,0.05)  
alpha_set = np.array([0, 0.25, 0.5, 1, 10, 30]).astype(float)
beta_set = np.array([0, 0.25, 0.5, 1, 10, 30]).astype(float)
sigmaF = 1
rho=0.2
phi = 0.2
ite=500
# sample_size= np.array([int(round(2**(7),0)),int(round(2**(7.5),0))])
###Modifying sample size 
# sample_size= np.array([int(round(2**(9.5),0))])
#initial one (full)
sample_size= np.array([int(round(2**(7),0)),int(round(2**(7.5),0)),int(round(2**(8),0)),int(round(2**(8.5),0)),int(round(2**(9),0)), int(round(2**(9.5),0))])

cum_errWeight_GL = np.zeros((len(sample_size),1))
cum_errWeight_FGL = np.zeros((len(sample_size),1))
cum_errWeight_EW = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_laplacian = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_l1 = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_grouplasso = np.zeros((len(sample_size),1))


cum_errTheta_GL = np.zeros((len(sample_size),1))
cum_errTheta_FGL = np.zeros((len(sample_size),1))
cum_errTheta_EW = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_laplacian = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_l1 = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_grouplasso = np.zeros((len(sample_size),1))


mc_errWeight_GL = np.zeros((ite,1))
mc_errWeight_FGL = np.zeros((ite,1))
mc_errWeight_EW = np.zeros((ite,1))
mc_errWeight_TVFGL_laplacian = np.zeros((ite,1))
mc_errWeight_TVFGL_l1 = np.zeros((ite,1))
mc_errWeight_TVFGL_grouplasso = np.zeros((ite,1))


mc_errTheta_GL = np.zeros((ite,1))
mc_errTheta_FGL = np.zeros((ite,1))
mc_errTheta_EW = np.zeros((ite,1))
mc_errTheta_TVFGL_laplacian = np.zeros((ite,1))
mc_errTheta_TVFGL_l1 = np.zeros((ite,1))
mc_errTheta_TVFGL_grouplasso = np.zeros((ite,1))

#####################################
gamma_n = np.zeros((len(sample_size),1))
gamma_reps = np.zeros((ite,1))
count=-1
for n in sample_size:
    count = count + 1
    p = int(round(n**(0.85),0) )
    #p=round((n[i])^(1.05))# HIGH DIMENSIONAL
    # p=round(3*(n[i])^(0.85))# HIGH DIMENSIONAL
    kDGP = math.ceil(2*(math.log2(p))**(1/2))
    k=kDGP
    kpoet = round(kDGP,0) 
    for jj in range(ite):
        print('n =', n, 'jj =', jj, 'out of', ite)
    ######################################
    ##########Dynamic Factor DGP##########
    ######################################   
    ###creating Toeplitz matrix and its Cholesky decomposition
    ###WITHOUT BREAK
        O = np.zeros((p, p))
        np.fill_diagonal(O, np.ones(p))     
        for h in range(1,p):
            np.fill_diagonal(O[h:], rho**np.repeat(h, p-h))
            np.fill_diagonal(O[:,h:], rho**np.repeat(h, p-h))
        
        Lambda = cholesky(O, lower=False)
        
    
        #######BEFORE THE REVISION: GENERATING FACTORS###############################
        F = np.zeros((n,kDGP))
        F[0,] = sigmaF*np.random.randn(1,kDGP)
        v=np.random.randn(n,kDGP)
        for j in range(0,kDGP):
            for t in range(1,n):
                F[t,j] = phi*F[t-1,j]+sigmaF*v[t,j]
        
        covf = np.cov(F, y=None, rowvar=False) #If rowvar is True (default), then each row represents a variable, with observations in the columns                     
        
        ##break points
        t1 = int(round(n/4))
        t2 = int(round(3*n/4))
        ##number of observations in the breaks        
        n1 = t1
        n2 = t2-t1
        n3 = n-n2-n1
        
        #################GENERATE EPS##################
        probability = 1- 500/((n)**(0.7)*(p)) #The probability that a coefficient is zero
        ##precision for regime 1
        Theta_u1 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.2, random_state=None)
        Sigma_u1 = inv(Theta_u1)
        ##precision for regime 2
        Theta_u2 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.4, random_state=None)
        Sigma_u2 = inv(Theta_u2)
        ##precision for regime 3
        Theta_u3 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.6, random_state=None)
        Sigma_u3 = inv(Theta_u3)

###########AFTER THE REVISION###############                
        eps1 = np.zeros((n1,p))
        eps2 = np.zeros((n2,p))
        eps3 = np.zeros((n3,p))
        breaks = np.zeros((n,1))
        mue = np.zeros((p,1))
        for jjj in range(0,n):
            if jjj <= t1:
                breaks[jjj,]=0
            if t1 < jjj <= t2:
                breaks[jjj,]=1 
            if jjj > t2:
                breaks[jjj,]=2
        for jjj in range(0,n1):
            eps1[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u1,1)
        for jjj in range(0,n2):
            eps2[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u2,1)
        for jjj in range(0,n3):
            eps3[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u3,1)
        
      
        breaks = breaks.astype(int)
        breaks = np.ravel(breaks)
        eps = np.concatenate((eps1, eps2, eps3), axis=0)
        cov_u1 = np.cov(eps1, y=None, rowvar=False) 
        Theta_u1 = np.linalg.inv(cov_u1)
        
        cov_u2 = np.cov(eps2, y=None, rowvar=False) 
        Theta_u2 = np.linalg.inv(cov_u2)
        
        cov_u3 = np.cov(eps3, y=None, rowvar=False) 
        Theta_u3 = np.linalg.inv(cov_u3)
        
    
        #################GENERATE RETURNS#################
        
        #########AFTER THE REVISION
        r = np.zeros((n,p))        
        r = F@Lambda[0:kDGP,] + eps

        ###################################
        ######BEFORE THE REVISION
        # r = F@Lambda[0:kDGP,] + eps
        # isig = Theta_u - Theta_u@Lambda[0:kDGP,].T@np.linalg.inv( np.linalg.inv(covf)+Lambda[0:kDGP,]@Theta_u@Lambda[0:kDGP,].T)@Lambda[0:kDGP,]@Theta_u
        
        
        #####AFTER THE REVISION####
        ##ON SECOND THOUGHT: how about just taking the inverse of returns?
        icov = np.cov(r, y=None, rowvar=False) 
        isig = np.linalg.inv(icov)
        ###two different population precision matrices before and after the break
        # isig1 = Theta_u1 - Theta_u1@Lambda1[0:kDGP,].T@np.linalg.inv( np.linalg.inv(covf)+Lambda1[0:kDGP,]@Theta_u1@Lambda1[0:kDGP,].T)@Lambda1[0:kDGP,]@Theta_u1
        # isig2 = Theta_u2 - Theta_u2@Lambda2[0:kDGP,].T@np.linalg.inv( np.linalg.inv(covf)+Lambda2[0:kDGP,]@Theta_u2@Lambda2[0:kDGP,].T)@Lambda2[0:kDGP,]@Theta_u2
        
      ###############################################################
       
        
        ##############Estimating USUAL factors and loadings (gamma=1)########
        L, V = np.linalg.eigh(np.dot(r.T, r))
        idx = L.argsort()[::-1]
        L = L[idx]  # eigenvalues, Nx1
        V = V[:, idx]  # eigenvectors columns, NxN
        lmb = V[:, 0:k]  # kx1
        Fhat = np.dot(r, lmb)  # Txr (r=1 for PC1)
        Y = r - Fhat@lmb.T ##these are the residuals
        
        covariate = Fhat
        betas = lmb.T

        ############Estimating precision matrix and combination weights using competing methods######
        ######## GLASSO###########
        GL = GraphicalLassoCV().fit(r)
        theta_GL = GL.get_precision()
        
        ########Factor GLASSO#######
        FGL = GraphicalLassoCV().fit(Y)
        theta_FGL_error = FGL.get_precision()
        
        if k==1:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
        else:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            
        ########Time-Varying TVFGL (gamma=1)#########
        ########Time-Varying TVFGL (gamma=1)#########
        denominator = p
        
        tuning_laplacian = CV_theta(penalty = '',Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)##laplacian (ridge) is defauls and was used at the beginning
        tuning_l1 = CV_theta(penalty = 'l1', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        tuning_grouplasso = CV_theta(penalty = 'l2', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
#         tuning_max = CV_theta(penalty = 'linf', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
      
        
        
#         tuning = CV_theta(Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)
        ###laplacian 
        tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############l1
        tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############group lasso
        tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
#         #############max norm
#         tvfgl = TimeGraphicalLasso(psi = 'linf', max_iter=100, alpha = tuning_max[0][0], beta = tuning_max[1][0]).fit(Y, breaks)
#         if k==1:
#             theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
#         else:
#             theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #################EQUALLY WEIGHTED########################
        ev = np.linalg.eig(np.cov(r, y=None, rowvar=False))
        eigenvalues = ev[0]
        mu=np.mean(eigenvalues)   
        CovshrIdent = np.zeros((p, p))
        np.fill_diagonal(CovshrIdent, np.repeat(mu, p)) 
        theta_EW=np.linalg.inv(CovshrIdent)
        
        
        ##################################################
        ######Estimation errors for Precision Matrix######
        ##################################################   
        errTheta_GL = math.log2(np.linalg.norm(theta_GL-isig, ord=2)/np.sqrt(denominator))
        errTheta_FGL = math.log2(np.linalg.norm(theta_FGL-isig, ord=2)/np.sqrt(denominator))
        errTheta_EW = math.log2(np.linalg.norm(theta_EW-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_laplacian = math.log2(np.linalg.norm(theta_TVFGL_laplacian-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_l1 = math.log2(np.linalg.norm(theta_TVFGL_l1-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_grouplasso = math.log2(np.linalg.norm(theta_TVFGL_grouplasso-isig, ord=2)/np.sqrt(denominator))
#         errTheta_TVFGL_max = math.log2(np.linalg.norm(theta_TVFGL_max-isig, ord=2)/np.sqrt(denominator))
        
        ##################################################
        ######Estimation errors for Portfolio Weights######
        ##################################################
        weight_true = portfolios(r,isig)[0]
        
        weight_GL = portfolios(r,theta_GL)[0]
        weight_FGL = portfolios(r,theta_FGL)[0]
        weight_EW = portfolios(r,theta_EW)[0]
        weight_TVFGL_laplacian = portfolios(r,theta_TVFGL_laplacian)[0]
        weight_TVFGL_l1 = portfolios(r,theta_TVFGL_l1)[0]
        weight_TVFGL_grouplasso = portfolios(r,theta_TVFGL_grouplasso)[0]
#         weight_TVFGL_max = portfolios(r,theta_TVFGL_max)[0]
#         weight_TVFGL_load = portfolios(r,theta_TVFGL_load)[0]
#         weight_TVFGL_load_zero = portfolios(r,theta_TVFGL_load_zero)[0]
        ############Global Minimum Variance (GMV)#########  
        errWeight_GL = math.log2(np.linalg.norm(weight_GL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_FGL = math.log2(np.linalg.norm(weight_FGL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_EW = math.log2(np.linalg.norm(weight_EW-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_laplacian = math.log2(np.linalg.norm(weight_TVFGL_laplacian-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_load = math.log2(np.linalg.norm(weight_TVFGL_load-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_load_zero = math.log2(np.linalg.norm(weight_TVFGL_load_zero-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_l1 = math.log2(np.linalg.norm(weight_TVFGL_l1-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_grouplasso = math.log2(np.linalg.norm(weight_TVFGL_grouplasso-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_max = math.log2(np.linalg.norm(weight_TVFGL_max-weight_true, ord=1)/np.sqrt(denominator))
    ############Precision Matrix#########  
        mc_errTheta_GL[jj] = errTheta_GL    
        mc_errTheta_FGL[jj] = errTheta_FGL
        mc_errTheta_EW[jj] = errTheta_EW
        mc_errTheta_TVFGL_laplacian[jj] = errTheta_TVFGL_laplacian
        mc_errTheta_TVFGL_l1[jj] = errTheta_TVFGL_l1
        mc_errTheta_TVFGL_grouplasso[jj] = errTheta_TVFGL_grouplasso
#         mc_errTheta_TVFGL_max[jj] = errTheta_TVFGL_max
#         mc_errTheta_TVFGL_load[jj] = errTheta_TVFGL_load
#         mc_errTheta_TVFGL_load_zero[jj] = errTheta_TVFGL_load_zero
        
    ############Global Minimum Variance (GMV)#########   
        mc_errWeight_GL[jj] = errWeight_GL    
        mc_errWeight_FGL[jj] = errWeight_FGL
        mc_errWeight_EW[jj] = errWeight_EW
        mc_errWeight_TVFGL_laplacian[jj] = errWeight_TVFGL_laplacian
        mc_errWeight_TVFGL_l1[jj] = errWeight_TVFGL_l1
        mc_errWeight_TVFGL_grouplasso[jj] = errWeight_TVFGL_grouplasso
#         mc_errWeight_TVFGL_max[jj] = errWeight_TVFGL_max
#         mc_errWeight_TVFGL_load[jj] = errWeight_TVFGL_load
#         mc_errWeight_TVFGL_load_zero[jj] = errWeight_TVFGL_load_zero
  ############Precision Matrix#########
    gamma_n[count] = np.mean(gamma_reps)
    
    cum_errTheta_GL[count] = np.mean(mc_errTheta_GL)
    cum_errTheta_FGL[count] = np.mean(mc_errTheta_FGL)
    cum_errTheta_EW[count] = np.mean(mc_errTheta_EW)
    cum_errTheta_TVFGL_laplacian[count] = np.mean(mc_errTheta_TVFGL_laplacian)
    cum_errTheta_TVFGL_l1[count] = np.mean(mc_errTheta_TVFGL_l1)
    cum_errTheta_TVFGL_grouplasso[count] = np.mean(mc_errTheta_TVFGL_grouplasso)
#     cum_errTheta_TVFGL_max[count] = np.mean(mc_errTheta_TVFGL_max)
#     cum_errTheta_TVFGL_load[count] = np.mean(mc_errTheta_TVFGL_load)
#     cum_errTheta_TVFGL_load_zero[count] = np.mean(mc_errTheta_TVFGL_load_zero)
    
    ############Global Minimum Variance (GMV)#########
    cum_errWeight_GL[count] = np.mean(mc_errWeight_GL)
    cum_errWeight_FGL[count] = np.mean(mc_errWeight_FGL)
    cum_errWeight_EW[count] = np.mean(mc_errWeight_EW)
    cum_errWeight_TVFGL_laplacian[count] = np.mean(mc_errWeight_TVFGL_laplacian)
    cum_errWeight_TVFGL_l1[count] = np.mean(mc_errWeight_TVFGL_l1)
    cum_errWeight_TVFGL_grouplasso[count] = np.mean(mc_errWeight_TVFGL_grouplasso)
#     cum_errWeight_TVFGL_max[count] = np.mean(mc_errWeight_TVFGL_max)
#     cum_errWeight_TVFGL_load[count] = np.mean(mc_errWeight_TVFGL_load)
#     cum_errWeight_TVFGL_load_zero[count] = np.mean(mc_errWeight_TVFGL_load_zero)
    
cum_errTheta = np.concatenate((cum_errTheta_FGL,cum_errTheta_GL,cum_errTheta_EW,cum_errTheta_TVFGL_laplacian,cum_errTheta_TVFGL_l1,cum_errTheta_TVFGL_grouplasso), axis=1)
savetxt('cum_errTheta_2breaks_theta.csv', cum_errTheta, delimiter=',')

cum_errWeight = np.concatenate((cum_errWeight_FGL,cum_errWeight_GL,cum_errWeight_EW,cum_errWeight_TVFGL_laplacian,cum_errWeight_TVFGL_l1,cum_errWeight_TVFGL_grouplasso), axis=1)
savetxt('cum_errWeight_2breaks_theta.csv', cum_errWeight, delimiter=',')

# savetxt('gamma_2breaks_theta.csv', gamma_n, delimiter=',')
      
 

In [ ]:
cum_errWeight_GL = np.zeros((len(sample_size),1))
cum_errWeight_FGL = np.zeros((len(sample_size),1))
cum_errWeight_EW = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_laplacian = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_l1 = np.zeros((len(sample_size),1))
cum_errWeight_TVFGL_grouplasso = np.zeros((len(sample_size),1))

cum_errTheta_GL = np.zeros((len(sample_size),1))
cum_errTheta_FGL = np.zeros((len(sample_size),1))
cum_errTheta_EW = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_laplacian = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_l1 = np.zeros((len(sample_size),1))
cum_errTheta_TVFGL_grouplasso = np.zeros((len(sample_size),1))

mc_errWeight_GL = np.zeros((ite,1))
mc_errWeight_FGL = np.zeros((ite,1))
mc_errWeight_EW = np.zeros((ite,1))
mc_errWeight_TVFGL_laplacian = np.zeros((ite,1))
mc_errWeight_TVFGL_l1 = np.zeros((ite,1))
mc_errWeight_TVFGL_grouplasso = np.zeros((ite,1))

mc_errTheta_GL = np.zeros((ite,1))
mc_errTheta_FGL = np.zeros((ite,1))
mc_errTheta_EW = np.zeros((ite,1))
mc_errTheta_TVFGL_laplacian = np.zeros((ite,1))
mc_errTheta_TVFGL_l1 = np.zeros((ite,1))
mc_errTheta_TVFGL_grouplasso = np.zeros((ite,1))

In [ ]:
#####Appendix Figure E8: TIME-VARYING PRECISION (fixed loadings): exploring different break magnitudes
count=-1
for n in sample_size:
    count = count + 1
    p = int(round(n**(0.85),0) )
    #p=round((n[i])^(1.05))# HIGH DIMENSIONAL
    # p=round(3*(n[i])^(0.85))# HIGH DIMENSIONAL
    kDGP = math.ceil(2*(math.log2(p))**(1/2))
    k=kDGP
    kpoet = round(kDGP,0) 
    for jj in range(ite):
        print('n =', n, 'jj =', jj, 'out of', ite)
    ######################################
    ##########Dynamic Factor DGP##########
    ######################################   
    ###creating Toeplitz matrix and its Cholesky decomposition
    ###WITHOUT BREAK
        O = np.zeros((p, p))
        np.fill_diagonal(O, np.ones(p))     
        for h in range(1,p):
            np.fill_diagonal(O[h:], rho**np.repeat(h, p-h))
            np.fill_diagonal(O[:,h:], rho**np.repeat(h, p-h))
        
        Lambda = cholesky(O, lower=False)
        
    
        #######BEFORE THE REVISION: GENERATING FACTORS###############################
        F = np.zeros((n,kDGP))
        F[0,] = sigmaF*np.random.randn(1,kDGP)
        v=np.random.randn(n,kDGP)
        for j in range(0,kDGP):
            for t in range(1,n):
                F[t,j] = phi*F[t-1,j]+sigmaF*v[t,j]
        
        covf = np.cov(F, y=None, rowvar=False) #If rowvar is True (default), then each row represents a variable, with observations in the columns                     
        
        n1 = int(round(n/2))
        n2 = n - n1
        #################GENERATE EPS##################
        probability = 1- 500/((n)**(0.7)*(p)) #The probability that a coefficient is zero
        ##precision for regime 1
        Theta_u1 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.4, random_state=None)
        Sigma_u1 = inv(Theta_u1)
        ##precision for regime 2
        Theta_u2 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.45, random_state=None)
        Sigma_u2 = inv(Theta_u2)
        

###########AFTER THE REVISION###############                
        eps1 = np.zeros((n1,p))
        eps2 = np.zeros((n2,p))
        breaks = np.zeros((n,1))
        mue = np.zeros((p,1))
        for jjj in range(0,n):
            if jjj <= n1:
                breaks[jjj,]=0
            else:
                breaks[jjj,]=1
        for jjj in range(0,n1):
            eps1[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u1,1)
        for jjj in range(0,n2):
            eps2[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u2,1)
      
        breaks = breaks.astype(int)
        breaks = np.ravel(breaks)
        eps = np.concatenate((eps1, eps2), axis=0)
        cov_u1 = np.cov(eps1, y=None, rowvar=False) 
        Theta_u1 = np.linalg.inv(cov_u1)
        
        cov_u2 = np.cov(eps2, y=None, rowvar=False) 
        Theta_u2 = np.linalg.inv(cov_u2)
        
    
        #################GENERATE RETURNS#################
        
        #########AFTER THE REVISION
        r = np.zeros((n,p))        
        r = F@Lambda[0:kDGP,] + eps

        ###################################
        ######BEFORE THE REVISION
        # r = F@Lambda[0:kDGP,] + eps
        # isig = Theta_u - Theta_u@Lambda[0:kDGP,].T@np.linalg.inv( np.linalg.inv(covf)+Lambda[0:kDGP,]@Theta_u@Lambda[0:kDGP,].T)@Lambda[0:kDGP,]@Theta_u
        
        
        #####AFTER THE REVISION####
        ##ON SECOND THOUGHT: how about just taking the inverse of returns?
        icov = np.cov(r, y=None, rowvar=False) 
        isig = np.linalg.inv(icov)
        ###two different population precision matrices before and after the break
        # isig1 = Theta_u1 - Theta_u1@Lambda1[0:kDGP,].T@np.linalg.inv( np.linalg.inv(covf)+Lambda1[0:kDGP,]@Theta_u1@Lambda1[0:kDGP,].T)@Lambda1[0:kDGP,]@Theta_u1
        # isig2 = Theta_u2 - Theta_u2@Lambda2[0:kDGP,].T@np.linalg.inv( np.linalg.inv(covf)+Lambda2[0:kDGP,]@Theta_u2@Lambda2[0:kDGP,].T)@Lambda2[0:kDGP,]@Theta_u2
        
      ###############################################################
       
        
        ##############Estimating USUAL factors and loadings (gamma=1)########
        L, V = np.linalg.eigh(np.dot(r.T, r))
        idx = L.argsort()[::-1]
        L = L[idx]  # eigenvalues, Nx1
        V = V[:, idx]  # eigenvectors columns, NxN
        lmb = V[:, 0:k]  # kx1
        Fhat = np.dot(r, lmb)  # Txr (r=1 for PC1)
        Y = r - Fhat@lmb.T ##these are the residuals
        
        covariate = Fhat
        betas = lmb.T

        ############Estimating precision matrix and combination weights using competing methods######
        ######## GLASSO###########
        GL = GraphicalLassoCV().fit(r)
        theta_GL = GL.get_precision()
        
        ########Factor GLASSO#######
        FGL = GraphicalLassoCV().fit(Y)
        theta_FGL_error = FGL.get_precision()
        
        if k==1:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
        else:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            
        ########Time-Varying TVFGL (gamma=1)#########
        ########Time-Varying TVFGL (gamma=1)#########
        denominator = p
        
        tuning_laplacian = CV_theta(penalty = '',Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)##laplacian (ridge) is defauls and was used at the beginning
        tuning_l1 = CV_theta(penalty = 'l1', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        tuning_grouplasso = CV_theta(penalty = 'l2', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
#         tuning_max = CV_theta(penalty = 'linf', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
      
        
        
#         tuning = CV_theta(Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)
        ###laplacian 
        tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############l1
        tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############group lasso
        tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
#         #############max norm
#         tvfgl = TimeGraphicalLasso(psi = 'linf', max_iter=100, alpha = tuning_max[0][0], beta = tuning_max[1][0]).fit(Y, breaks)
#         if k==1:
#             theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
#         else:
#             theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #################EQUALLY WEIGHTED########################
        ev = np.linalg.eig(np.cov(r, y=None, rowvar=False))
        eigenvalues = ev[0]
        mu=np.mean(eigenvalues)   
        CovshrIdent = np.zeros((p, p))
        np.fill_diagonal(CovshrIdent, np.repeat(mu, p)) 
        theta_EW=np.linalg.inv(CovshrIdent)
        
        
        ##################################################
        ######Estimation errors for Precision Matrix######
        ##################################################   
        errTheta_GL = math.log2(np.linalg.norm(theta_GL-isig, ord=2)/np.sqrt(denominator))
        errTheta_FGL = math.log2(np.linalg.norm(theta_FGL-isig, ord=2)/np.sqrt(denominator))
        errTheta_EW = math.log2(np.linalg.norm(theta_EW-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_laplacian = math.log2(np.linalg.norm(theta_TVFGL_laplacian-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_l1 = math.log2(np.linalg.norm(theta_TVFGL_l1-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_grouplasso = math.log2(np.linalg.norm(theta_TVFGL_grouplasso-isig, ord=2)/np.sqrt(denominator))
#         errTheta_TVFGL_max = math.log2(np.linalg.norm(theta_TVFGL_max-isig, ord=2)/np.sqrt(denominator))
        
        ##################################################
        ######Estimation errors for Portfolio Weights######
        ##################################################
        weight_true = portfolios(r,isig)[0]
        
        weight_GL = portfolios(r,theta_GL)[0]
        weight_FGL = portfolios(r,theta_FGL)[0]
        weight_EW = portfolios(r,theta_EW)[0]
        weight_TVFGL_laplacian = portfolios(r,theta_TVFGL_laplacian)[0]
        weight_TVFGL_l1 = portfolios(r,theta_TVFGL_l1)[0]
        weight_TVFGL_grouplasso = portfolios(r,theta_TVFGL_grouplasso)[0]
#         weight_TVFGL_max = portfolios(r,theta_TVFGL_max)[0]
#         weight_TVFGL_load = portfolios(r,theta_TVFGL_load)[0]
#         weight_TVFGL_load_zero = portfolios(r,theta_TVFGL_load_zero)[0]
        ############Global Minimum Variance (GMV)#########  
        errWeight_GL = math.log2(np.linalg.norm(weight_GL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_FGL = math.log2(np.linalg.norm(weight_FGL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_EW = math.log2(np.linalg.norm(weight_EW-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_laplacian = math.log2(np.linalg.norm(weight_TVFGL_laplacian-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_load = math.log2(np.linalg.norm(weight_TVFGL_load-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_load_zero = math.log2(np.linalg.norm(weight_TVFGL_load_zero-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_l1 = math.log2(np.linalg.norm(weight_TVFGL_l1-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_grouplasso = math.log2(np.linalg.norm(weight_TVFGL_grouplasso-weight_true, ord=1)/np.sqrt(denominator))
#         errWeight_TVFGL_max = math.log2(np.linalg.norm(weight_TVFGL_max-weight_true, ord=1)/np.sqrt(denominator))
    ############Precision Matrix#########  
        mc_errTheta_GL[jj] = errTheta_GL    
        mc_errTheta_FGL[jj] = errTheta_FGL
        mc_errTheta_EW[jj] = errTheta_EW
        mc_errTheta_TVFGL_laplacian[jj] = errTheta_TVFGL_laplacian
        mc_errTheta_TVFGL_l1[jj] = errTheta_TVFGL_l1
        mc_errTheta_TVFGL_grouplasso[jj] = errTheta_TVFGL_grouplasso
#         mc_errTheta_TVFGL_max[jj] = errTheta_TVFGL_max
#         mc_errTheta_TVFGL_load[jj] = errTheta_TVFGL_load
#         mc_errTheta_TVFGL_load_zero[jj] = errTheta_TVFGL_load_zero
        
    ############Global Minimum Variance (GMV)#########   
        mc_errWeight_GL[jj] = errWeight_GL    
        mc_errWeight_FGL[jj] = errWeight_FGL
        mc_errWeight_EW[jj] = errWeight_EW
        mc_errWeight_TVFGL_laplacian[jj] = errWeight_TVFGL_laplacian
        mc_errWeight_TVFGL_l1[jj] = errWeight_TVFGL_l1
        mc_errWeight_TVFGL_grouplasso[jj] = errWeight_TVFGL_grouplasso
#         mc_errWeight_TVFGL_max[jj] = errWeight_TVFGL_max
#         mc_errWeight_TVFGL_load[jj] = errWeight_TVFGL_load
#         mc_errWeight_TVFGL_load_zero[jj] = errWeight_TVFGL_load_zero
  ############Precision Matrix#########
#     gamma_n[count] = np.mean(gamma_reps)
    
    cum_errTheta_GL[count] = np.mean(mc_errTheta_GL)
    cum_errTheta_FGL[count] = np.mean(mc_errTheta_FGL)
    cum_errTheta_EW[count] = np.mean(mc_errTheta_EW)
    cum_errTheta_TVFGL_laplacian[count] = np.mean(mc_errTheta_TVFGL_laplacian)
    cum_errTheta_TVFGL_l1[count] = np.mean(mc_errTheta_TVFGL_l1)
    cum_errTheta_TVFGL_grouplasso[count] = np.mean(mc_errTheta_TVFGL_grouplasso)
#     cum_errTheta_TVFGL_max[count] = np.mean(mc_errTheta_TVFGL_max)
#     cum_errTheta_TVFGL_load[count] = np.mean(mc_errTheta_TVFGL_load)
#     cum_errTheta_TVFGL_load_zero[count] = np.mean(mc_errTheta_TVFGL_load_zero)
    
    ############Global Minimum Variance (GMV)#########
    cum_errWeight_GL[count] = np.mean(mc_errWeight_GL)
    cum_errWeight_FGL[count] = np.mean(mc_errWeight_FGL)
    cum_errWeight_EW[count] = np.mean(mc_errWeight_EW)
    cum_errWeight_TVFGL_laplacian[count] = np.mean(mc_errWeight_TVFGL_laplacian)
    cum_errWeight_TVFGL_l1[count] = np.mean(mc_errWeight_TVFGL_l1)
    cum_errWeight_TVFGL_grouplasso[count] = np.mean(mc_errWeight_TVFGL_grouplasso)
#     cum_errWeight_TVFGL_max[count] = np.mean(mc_errWeight_TVFGL_max)
#     cum_errWeight_TVFGL_load[count] = np.mean(mc_errWeight_TVFGL_load)
#     cum_errWeight_TVFGL_load_zero[count] = np.mean(mc_errWeight_TVFGL_load_zero)
    
cum_errTheta = np.concatenate((cum_errTheta_FGL,cum_errTheta_GL,cum_errTheta_EW,cum_errTheta_TVFGL_laplacian,cum_errTheta_TVFGL_l1,cum_errTheta_TVFGL_grouplasso), axis=1)
savetxt('cum_errTheta_breaksmall_theta.csv', cum_errTheta, delimiter=',')

cum_errWeight = np.concatenate((cum_errWeight_FGL,cum_errWeight_GL,cum_errWeight_EW,cum_errWeight_TVFGL_laplacian,cum_errWeight_TVFGL_l1,cum_errWeight_TVFGL_grouplasso), axis=1)
savetxt('cum_errWeight_breaksmall_theta.csv', cum_errWeight, delimiter=',')

# savetxt('gamma_break_theta.csv', gamma_n, delimiter=',')
      
 

In [ ]:
#####Appendix Figure E1: TIME-VARYING PRECISION (fixed loadings) AND DIFFERENT PENALTY FUNCTIONS
count=-1
for n in sample_size:
    count = count + 1
    p = int(round(n**(0.85),0) )
    #p=round((n[i])^(1.05))# HIGH DIMENSIONAL
    # p=round(3*(n[i])^(0.85))# HIGH DIMENSIONAL
    kDGP = math.ceil(2*(math.log2(p))**(1/2))
    k=kDGP
    kpoet = round(kDGP,0) 
    for jj in range(ite):
        print('n =', n,'jj =', jj)
######################################
    ##########Dynamic Factor DGP##########
    ######################################   
    ###creating Toeplitz matrix and its Cholesky decomposition        
        #######GENERATING LOADINGS##############################
        ####need to create two Lambda matrices associated with different rhos
        ###BEFORE BREAK 1
        rho1 = 0.2
        O1 = np.zeros((p, p))
        np.fill_diagonal(O1, np.ones(p))     
        for h in range(1,p):
            np.fill_diagonal(O1[h:], rho1**np.repeat(h, p-h))
            np.fill_diagonal(O1[:,h:], rho1**np.repeat(h, p-h))
        
        Lambda1 = cholesky(O1, lower=False)
        
        ###AFTER BREAK 1
        rho2 = 0.6
        O2 = np.zeros((p, p))
        np.fill_diagonal(O2, np.ones(p))     
        for h in range(1,p):
            np.fill_diagonal(O2[h:], rho2**np.repeat(h, p-h))
            np.fill_diagonal(O2[:,h:], rho2**np.repeat(h, p-h))
        
        Lambda2 = cholesky(O2, lower=False)
    
        ####AFTER THE REVISION: ALTERNATIVE WAY OF GENERATING FACTORS
        n1 = int(round(n/2))
        n2 = n - n1
        F1 = np.zeros((n1,kDGP))
        F1[0,] = sigmaF*np.random.randn(1,kDGP)
        v=np.random.randn(n1,kDGP)
        for j in range(0,kDGP):
            for t in range(1,n1):
                F1[t,j] = phi*F1[t-1,j]+sigmaF*v[t,j]
                
                
        F2 = np.zeros((n2,kDGP))
        F2[0,] = sigmaF*np.random.randn(1,kDGP)
        v=np.random.randn(n2,kDGP)
        for j in range(0,kDGP):
            for t in range(1,n2):
                F2[t,j] = phi*F2[t-1,j]+sigmaF*v[t,j]        
        
        F = np.concatenate((F1, F2), axis=0)
        covf = np.cov(F, y=None, rowvar=False) #If rowvar is True (default), then each row represents a variable, with observations in the columns                     
        
        
        #################GENERATE EPS##################
        probability = 1- 500/((n)**(0.7)*(p)) #The probability that a coefficient is zero
        ##precision for regime 1
        Theta_u1 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.4, random_state=None)
        Sigma_u1 = inv(Theta_u1)
        ##precision for regime 2
        Theta_u2 = sklearn.datasets.make_sparse_spd_matrix(dim=p, alpha=probability, norm_diag=False, smallest_coef=0.1, largest_coef=0.6, random_state=None)
        Sigma_u2 = inv(Theta_u2)

###########AFTER THE REVISION###############                
        eps1 = np.zeros((n1,p))
        eps2 = np.zeros((n2,p))
        breaks = np.zeros((n,1))
        mue = np.zeros((p,1))
        for jjj in range(0,n):
            if jjj <= n1:
                breaks[jjj,]=0
            else:
                breaks[jjj,]=1
        for jjj in range(0,n1):
            eps1[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u1,1)
        for jjj in range(0,n2):
            eps2[jjj,]=np.random.multivariate_normal(mue.ravel(),Sigma_u1,1)
      
        breaks = breaks.astype(int)
        breaks = np.ravel(breaks)
        eps = np.concatenate((eps1, eps2), axis=0)
        cov_u1 = np.cov(eps1, y=None, rowvar=False) 
        Theta_u1 = np.linalg.inv(cov_u1)
        
        cov_u2 = np.cov(eps2, y=None, rowvar=False) 
        Theta_u2 = np.linalg.inv(cov_u2)
        
        #########AFTER THE REVISION
        r1 = np.zeros((n1,p))
        r2 = np.zeros((n2,p))
        
        r1 = F1@Lambda1[0:kDGP,] + eps1
        r2 = F2@Lambda2[0:kDGP,] + eps2
        
        r = np.concatenate((r1, r2), axis=0)
        ###################################
        #####AFTER THE REVISION####
        icov = np.cov(r, y=None, rowvar=False) 
        isig = np.linalg.inv(icov)
        
        
      ###Incorporating info about time-varying factor loadings
 
      ###############################################################
        ##############Estimating factors and loadings for time-varying########
        gamma_opt = CV_gamma(gamma_set,r, r1, r2, k)
        print('gamma_opt =', gamma_opt)
        gamma_reps[jj] = gamma_opt
        ####modified returns for time-varying loadings only!!!
        r_load = r.copy()
        for row in range(r_load.shape[0]):
            for col in range(r_load.shape[1]):
                if row <= n1:
                    r_load[row,col]=gamma_opt* r_load[row,col] 

        L_load, V_load = np.linalg.eigh(np.dot(r_load.T, r_load))
        idx_load = L_load.argsort()[::-1]
        L_load = L_load[idx_load]  # eigenvalues, Nx1
        V_load = V_load[:, idx_load]  # eigenvectors columns, NxN
        lmb_load = V_load[:, 0:k]  # kx1
        # Fhat = np.dot(r, lmb)  # Txr (r=1 for PC1)
        ###According to Su (2017, JoE) if we obtain Fhat
        ###as usual they are only consistent for a rotational version
        ###hence, to get a consistent estimator use a two-stage procedure (OLS)
        Fhat_load = r@lmb_load@np.linalg.inv(lmb_load.T@lmb_load)
        Y_load = r - Fhat_load@lmb_load.T ##these are the residuals
        
        covariate_load = Fhat_load
        betas_load = lmb_load.T
        
        
        ##############Estimating USUAL factors and loadings (gamma=1)########
        L, V = np.linalg.eigh(np.dot(r.T, r))
        idx = L.argsort()[::-1]
        L = L[idx]  # eigenvalues, Nx1
        V = V[:, idx]  # eigenvectors columns, NxN
        lmb = V[:, 0:k]  # kx1
        Fhat = np.dot(r, lmb)  # Txr (r=1 for PC1)
        Y = r - Fhat@lmb.T ##these are the residuals
        
        covariate = Fhat
        betas = lmb.T

        ############Estimating precision matrix and combination weights using competing methods######
        ######## GLASSO###########
        GL = GraphicalLassoCV().fit(r)
        theta_GL = GL.get_precision()
        
        ########Factor GLASSO#######
        FGL = GraphicalLassoCV().fit(Y)
        theta_FGL_error = FGL.get_precision()
        
        if k==1:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
        else:
            theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            
        ########Time-Varying TVFGL (gamma=1)#########
        denominator = p
        
        tuning_laplacian = CV_theta(penalty = '',Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)##laplacian (ridge) is defauls and was used at the beginning
        tuning_l1 = CV_theta(penalty = 'l1', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        tuning_grouplasso = CV_theta(penalty = 'l2', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
#         tuning_max = CV_theta(penalty = 'linf', Y=Y,breaks = breaks, betas = betas, covariate=covariate, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
      
        print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)
        ###laplacian 
        tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############l1
        tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
        
        #############group lasso
        tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y, breaks)
        if k==1:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
                
         ########Time-Varying TVFGL(BOTH precision and loadings time-varying)#########        
        tuning_laplacian_load = CV_theta(penalty = '',Y=Y_load,breaks = breaks, betas = betas_load, covariate=covariate_load, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)##laplacian (ridge) is defauls and was used at the beginning
        tuning_l1_load = CV_theta(penalty = 'l1', Y=Y_load,breaks = breaks, betas = betas_load, covariate=covariate_load, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
        tuning_grouplasso_load = CV_theta(penalty = 'l2', Y=Y_load,breaks = breaks, betas = betas_load, covariate=covariate_load, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
#         tuning_max = CV_theta(penalty = 'linf', Y=Y_load,breaks = breaks, betas = betas_load, covariate=covariate_load, alpha_set=alpha_set, beta_set=beta_set, isig=isig, denominator=denominator)
      
        print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)

        ###laplacian       
        tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y_load, breaks)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
        if k==1:
            theta_TVFGL_laplacian_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_laplacian_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
     
        ###l1       
        tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y_load, breaks)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
        if k==1:
            theta_TVFGL_l1_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_l1_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
     ###############################################################   
        ###group lasso     
        tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y_load, breaks)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
        if k==1:
            theta_TVFGL_grouplasso_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
        else:
            theta_TVFGL_grouplasso_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
     ###############################################################   
        
        
        
        #################EQUALLY WEIGHTED########################
        ev = np.linalg.eig(np.cov(r, y=None, rowvar=False))
        eigenvalues = ev[0]
        mu=np.mean(eigenvalues)   
        CovshrIdent = np.zeros((p, p))
        np.fill_diagonal(CovshrIdent, np.repeat(mu, p)) 
        theta_EW=np.linalg.inv(CovshrIdent)
        
        
        ##################################################
        ######Estimation errors for Precision Matrix######
        ##################################################   
        errTheta_GL = math.log2(np.linalg.norm(theta_GL-isig, ord=2)/np.sqrt(denominator))
        errTheta_FGL = math.log2(np.linalg.norm(theta_FGL-isig, ord=2)/np.sqrt(denominator))
        errTheta_EW = math.log2(np.linalg.norm(theta_EW-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_laplacian = math.log2(np.linalg.norm(theta_TVFGL_laplacian-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_l1 = math.log2(np.linalg.norm(theta_TVFGL_l1-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_grouplasso = math.log2(np.linalg.norm(theta_TVFGL_grouplasso-isig, ord=2)/np.sqrt(denominator))
#         errTheta_TVFGL_max = math.log2(np.linalg.norm(theta_TVFGL_max-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_laplacian_load = math.log2(np.linalg.norm(theta_TVFGL_laplacian_load-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_l1_load = math.log2(np.linalg.norm(theta_TVFGL_l1_load-isig, ord=2)/np.sqrt(denominator))
        errTheta_TVFGL_grouplasso_load = math.log2(np.linalg.norm(theta_TVFGL_grouplasso_load-isig, ord=2)/np.sqrt(denominator))
        ##################################################
        ######Estimation errors for Portfolio Weights######
        ##################################################
        weight_true = portfolios(r,isig)[0]
        
        weight_GL = portfolios(r,theta_GL)[0]
        weight_FGL = portfolios(r,theta_FGL)[0]
        weight_EW = portfolios(r,theta_EW)[0]
        weight_TVFGL_laplacian = portfolios(r,theta_TVFGL_laplacian)[0]
        weight_TVFGL_l1 = portfolios(r,theta_TVFGL_l1)[0]
        weight_TVFGL_grouplasso = portfolios(r,theta_TVFGL_grouplasso)[0]
        weight_TVFGL_laplacian_load = portfolios(r,theta_TVFGL_laplacian_load)[0]
        weight_TVFGL_l1_load = portfolios(r,theta_TVFGL_l1_load)[0]
        weight_TVFGL_grouplasso_load = portfolios(r,theta_TVFGL_grouplasso_load)[0]
        ############Global Minimum Variance (GMV)#########  
        errWeight_GL = math.log2(np.linalg.norm(weight_GL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_FGL = math.log2(np.linalg.norm(weight_FGL-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_EW = math.log2(np.linalg.norm(weight_EW-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_laplacian = math.log2(np.linalg.norm(weight_TVFGL_laplacian-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_l1 = math.log2(np.linalg.norm(weight_TVFGL_l1-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_grouplasso = math.log2(np.linalg.norm(weight_TVFGL_grouplasso-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_laplacian_load = math.log2(np.linalg.norm(weight_TVFGL_laplacian_load-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_l1_load = math.log2(np.linalg.norm(weight_TVFGL_l1_load-weight_true, ord=1)/np.sqrt(denominator))
        errWeight_TVFGL_grouplasso_load = math.log2(np.linalg.norm(weight_TVFGL_grouplasso_load-weight_true, ord=1)/np.sqrt(denominator))
        
    ############Precision Matrix#########  
        mc_errTheta_GL[jj] = errTheta_GL    
        mc_errTheta_FGL[jj] = errTheta_FGL
        mc_errTheta_EW[jj] = errTheta_EW
        mc_errTheta_TVFGL_laplacian[jj] = errTheta_TVFGL_laplacian
        mc_errTheta_TVFGL_l1[jj] = errTheta_TVFGL_l1
        mc_errTheta_TVFGL_grouplasso[jj] = errTheta_TVFGL_grouplasso
        mc_errTheta_TVFGL_laplacian_load[jj] = errTheta_TVFGL_laplacian_load
        mc_errTheta_TVFGL_l1_load[jj] = errTheta_TVFGL_l1_load
        mc_errTheta_TVFGL_grouplasso_load[jj] = errTheta_TVFGL_grouplasso_load
        
    ############Global Minimum Variance (GMV)#########   
        mc_errWeight_GL[jj] = errWeight_GL    
        mc_errWeight_FGL[jj] = errWeight_FGL
        mc_errWeight_EW[jj] = errWeight_EW
        mc_errWeight_TVFGL_laplacian[jj] = errWeight_TVFGL_laplacian
        mc_errWeight_TVFGL_l1[jj] = errWeight_TVFGL_l1
        mc_errWeight_TVFGL_grouplasso[jj] = errWeight_TVFGL_grouplasso
        mc_errWeight_TVFGL_laplacian_load[jj] = errWeight_TVFGL_laplacian_load
        mc_errWeight_TVFGL_l1_load[jj] = errWeight_TVFGL_l1_load
        mc_errWeight_TVFGL_grouplasso_load[jj] = errWeight_TVFGL_grouplasso_load
  ############Precision Matrix#########
    gamma_n[count] = np.mean(gamma_reps)
    
    cum_errTheta_GL[count] = np.mean(mc_errTheta_GL)
    cum_errTheta_FGL[count] = np.mean(mc_errTheta_FGL)
    cum_errTheta_EW[count] = np.mean(mc_errTheta_EW)
    cum_errTheta_TVFGL_laplacian[count] = np.mean(mc_errTheta_TVFGL_laplacian)
    cum_errTheta_TVFGL_l1[count] = np.mean(mc_errTheta_TVFGL_l1)
    cum_errTheta_TVFGL_grouplasso[count] = np.mean(mc_errTheta_TVFGL_grouplasso)
    cum_errTheta_TVFGL_laplacian_load[count] = np.mean(mc_errTheta_TVFGL_laplacian_load)
    cum_errTheta_TVFGL_l1_load[count] = np.mean(mc_errTheta_TVFGL_l1_load)
    cum_errTheta_TVFGL_grouplasso_load[count] = np.mean(mc_errTheta_TVFGL_grouplasso_load)
    
    ############Global Minimum Variance (GMV)#########
    cum_errWeight_GL[count] = np.mean(mc_errWeight_GL)
    cum_errWeight_FGL[count] = np.mean(mc_errWeight_FGL)
    cum_errWeight_EW[count] = np.mean(mc_errWeight_EW)
    cum_errWeight_TVFGL_laplacian[count] = np.mean(mc_errWeight_TVFGL_laplacian)
    cum_errWeight_TVFGL_l1[count] = np.mean(mc_errWeight_TVFGL_l1)
    cum_errWeight_TVFGL_grouplasso[count] = np.mean(mc_errWeight_TVFGL_grouplasso)
    cum_errWeight_TVFGL_laplacian_load[count] = np.mean(mc_errWeight_TVFGL_laplacian_load)
    cum_errWeight_TVFGL_l1_load[count] = np.mean(mc_errWeight_TVFGL_l1_load)
    cum_errWeight_TVFGL_grouplasso_load[count] = np.mean(mc_errWeight_TVFGL_grouplasso_load)
    
cum_errTheta = np.concatenate((cum_errTheta_FGL,cum_errTheta_GL,cum_errTheta_EW,cum_errTheta_TVFGL_laplacian,cum_errTheta_TVFGL_l1,cum_errTheta_TVFGL_grouplasso,cum_errTheta_TVFGL_laplacian_load,cum_errTheta_TVFGL_l1_load,cum_errTheta_TVFGL_grouplasso_load), axis=1)
savetxt('cum_errTheta_break_theta.csv', cum_errTheta, delimiter=',')

cum_errWeight = np.concatenate((cum_errWeight_FGL,cum_errWeight_GL,cum_errWeight_EW,cum_errWeight_TVFGL_laplacian,cum_errWeight_TVFGL_l1,cum_errWeight_TVFGL_grouplasso,cum_errWeight_TVFGL_laplacian_load,cum_errWeight_TVFGL_l1_load,cum_errWeight_TVFGL_grouplasso_load), axis=1)
savetxt('cum_errWeight_break_theta_and_loadings.csv', cum_errWeight, delimiter=',')

savetxt('gamma_break_theta_and_loadings.csv', gamma_n, delimiter=',')
      
 